In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb  # For å lese .dat og .hea filer


## Load the data
This dataset is very large, will take several hours to download.

In [ ]:
!curl -O https://physionet.org/static/published-projects/ptb-xl/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  1 1754M    1 24.7M    0     0   222k      0  2:14:51  0:01:54  2:12:57  236k

In [ ]:
!unzip -q ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3.zip
!ls